In [274]:
# Execute if necessary
# %%capture
# !pip install numpy seaborn matplotlib pandas openml

In [275]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List
import openml
import sklearn
from tqdm import tqdm

# Práctica 5: Introducción a Modelos de Clasificación

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones.

La entrega de la práctica será en la siguiente sesión a menos que la indicación sea otra. La revisión iniciará al iniciar la sesión y únicamente podrá ser evaluada durante la duración de la sesión.

## Asignación 1

Implemente una clase que permita ajustar un modelo KNN. La clase debe cumplir con los siguientes requisitos:

- Debe contar con un método para realizar el entrenamiento en caso de ser requerido.
- Debe contar con un método para realizar las predicciones.
- Cada método debe imprimir una barra de progreso que permita conocer el tiempo estimado en que terminará.
- La clase debe permitir el uso de la distancia Mikownski y similitud coseno.

In [276]:
# A class that implement a knn classifier
class KNNClassifier:
    def __init__(self, k: int = 1):
        self.k = k
        self.X = None
        self.y = None

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        self.X = X
        self.y = y

    def predict(self, X: np.ndarray, distance: str = "cos", p: int = 2) -> np.ndarray:
        dist = []
        # Calculate the distance between each test point and all training points
        if distance == "cos":
            dist = sklearn.metrics.pairwise.cosine_distances(X, self.X)
        elif distance == "mink":
            for i in tqdm(range(X.shape[0]), desc ="Calculating distances"):
                dist.append(self.minkowski_distance(X[i], self.X, p))

        # Find the k nearest points
        nearest = np.argsort(dist, axis=1)[:, :self.k]
        
        # Find the most common class among the k nearest points
        common_classes = []
        for i in tqdm(range(len(nearest)), desc ="Finding most common class"):
            common_classes.append(np.argmax(np.bincount(self.y[nearest[i]])))
        return common_classes
    
    # Mikownski distance function
    def minkowski_distance(self, x: np.ndarray, y: np.ndarray, p: int) -> float:
        return sum(abs(e1-e2)**p for e1, e2 in zip(x,y))**(1/p)
    
    # cosine similarity function
    def cosine_distance(self, x: np.ndarray, y: np.ndarray) -> float:
        return np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

## Asignación 2

Implemente una clase que permita ajustar un modelo Naive Bayes. La clase debe cumplir con los siguientes requisitos:

- Debe contar con un método para realizar el entrenamiento en caso de ser requerido.
- Debe contar con un método para realizar las predicciones.
- Cada método debe imprimir una barra de progreso que permita conocer el tiempo estimado en que terminará.

In [277]:
# A class that implement a naive bayes classifier
class NaiveBayesClassifier:
    def __init__(self):
        self.X = None
        self.y = None
        self.prior = None
        self.likelihood = None

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        # Example of X: [[1, 0, 1, 1], [0, 1, 0, 1], [1, 1, 1, 0]]
        # Example of y: [0, 1, 0]
        self.X = X
        self.y = y

        # Calculate the prior probability of each class
        self.prior = np.bincount(y) / len(y)

        # Calculate the likelihood of each feature given each class
        self.likelihood = np.zeros((len(np.unique(y)), X.shape[1]))
        for i in tqdm(range(len(np.unique(y))), desc ="Calculating likelihood"):
            self.likelihood[i] = np.mean(X[y == i], axis=0)

    def predict(self, X: np.ndarray) -> np.ndarray:
        # Calculate the posterior probability of each class given the features
        posterior = np.zeros((len(X), len(self.prior)))
        for i in tqdm(range(len(X)), desc ="Calculating probabilities"):
            for j in range(len(self.prior)):
                posterior[i][j] = np.prod((self.likelihood[j] ** X[i]) * (1 - self.likelihood[j]) ** (1 - X[i])) * self.prior[j]

        # Return the most probable class
        return np.argmax(posterior, axis=1)

In [278]:
# An example using the knn classifier
def knn_example():
    # Load the dataset
    data = openml.datasets.get_dataset(31)
    X, y, categorical_indicator, attribute_names = data.get_data(
        dataset_format="array", target=data.default_target_attribute
    )

    # Split the dataset into train and test
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, random_state=1
    )

    # Create the knn classifier
    knn = KNNClassifier(k=3)

    # Fit the model
    knn.fit(X_train, y_train)

    # Predict the first 2 features of each test point
    y_pred = knn.predict(X_test, distance="mink")

    # Calculate the accuracy
    print(f"Accuracy: {np.sum(y_pred == y_test) / len(y_test)}")
    print('a')

knn_example()

Finding most common class: 100%|██████████| 250/250 [00:00<00:00, 249660.95it/s]

Accuracy: 0.704
a


In [279]:
# Ecample using the naive bayes classifier
def naive_bayes_example():
    # Load the dataset
    data = openml.datasets.get_dataset(31)
    X, y, categorical_indicator, attribute_names = data.get_data(
        dataset_format="array", target=data.default_target_attribute
    )

    # Split the dataset into train and test
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, random_state=1
    )

    # Create the naive bayes classifier
    nb = NaiveBayesClassifier()

    # Fit the model
    nb.fit(X_train, y_train)

    # Predict the class of each test point
    y_pred = nb.predict(X_test)

    # Calculate the accuracy
    print(f"Accuracy: {np.sum(y_pred == y_test) / len(y_test)}")

naive_bayes_example()

Calculating probabilities:   0%|          | 0/250 [00:00<?, ?it/s]C:\Users\Iván\AppData\Local\Temp\ipykernel_4308\2794424327.py:28: RuntimeWarning: overflow encountered in power
  posterior[i][j] = np.prod((self.likelihood[j] ** X[i]) * (1 - self.likelihood[j]) ** (1 - X[i])) * self.prior[j]
C:\Users\Iván\AppData\Local\Temp\ipykernel_4308\2794424327.py:28: RuntimeWarning: invalid value encountered in multiply
  posterior[i][j] = np.prod((self.likelihood[j] ** X[i]) * (1 - self.likelihood[j]) ** (1 - X[i])) * self.prior[j]
Calculating probabilities: 100%|██████████| 250/250 [00:00<00:00, 41641.56it/s]

Accuracy: 0.704


En esta sección se incluye un dataset real. El dataset importado se llama _spambase_ y puede leer su descripción en la siguiente liga

https://www.openml.org/d/42904

In [280]:
# Descarga la metadata del dataset
dataset_info = openml.datasets.get_dataset(42904, download_data=False)

target = "CLASS"

(
    features, # Dataframe con las características que se pueden utilizar para predecir
    outputs, # Columna a predecir
    _, # Máscara que indica que columnas de todas las características son categoricas
    columns # Lista con el nombre de las características
)= dataset_info.get_data(
    dataset_format="dataframe", target=target
)

columns = np.array(columns)

In [281]:
print(f"La columna a predecir se llama '{target}'")
print(f"Todas las características son {str(columns)}")

La columna a predecir se llama 'CLASS'
Todas las características son ['COMMENT_ID' 'AUTHOR' 'DATE' 'CONTENT']


In [282]:
# Impresión de las características
features

,COMMENT_ID,AUTHOR,DATE,CONTENT
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ..."
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿
...,...,...,...,...
345,z13th1q4yzihf1bll23qxzpjeujterydj,Carmen Racasanu,2014-11-14T13:27:52,How can this have 2 billion views when there's...
346,z13fcn1wfpb5e51xe04chdxakpzgchyaxzo0k,diego mogrovejo,2014-11-14T13:28:08,I don't now why I'm watching this in 2014﻿
347,z130zd5b3titudkoe04ccbeohojxuzppvbg,BlueYetiPlayz -Call Of Duty and More,2015-05-23T13:04:32,subscribe to me for call of duty vids and give...
348,z12he50arvrkivl5u04cctawgxzkjfsjcc4,Photo Editor,2015-06-05T14:14:48,hi guys please my android photo editor downloa...


In [283]:
outputs

0      1
1      1
2      1
3      1
4      1
      ..
345    0
346    0
347    1
348    1
349    0
Name: CLASS, Length: 350, dtype: uint8

## Asignación 3

Realice el preprocesamiento que considere adecuado para que el dataset pueda ser procesado por un modelo de clasificación. __No olvide utilizar particionar en entrenamiento y validación, y realizar el preprocesamiento de manera adecuada__.

## Asignación 4

Realice una visualización de los datos en dos dimensiones. Posteriormente entrene un modelo de KNN (utilizando la clase implementada anteriormente) y valide que su modelo generaliza bien.

## Asignación 5

Entrene un modelo de Naive Bayes (utilizando la clase implementada anteriormente) y valide que su modelo generaliza bien.